In [1]:
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio.features import shapes
import os
import requests
import zipfile

In [2]:
file_path = './heat_risk_analysis_Day+1_20240723.geoparquet'

In [3]:
gdf = gpd.read_parquet(file_path)

In [4]:
gdf.head()

geometry  raster_value  \
0  POLYGON ((-106.44867 49.0167, -106.44867 48.98...           2.0   
1  POLYGON ((-123.08047 49.00174, -123.08047 48.9...           1.0   
2  POLYGON ((-111.44505 49.00174, -111.44505 48.9...           2.0   
3  POLYGON ((-111.14846 49.00174, -111.14846 48.9...           2.0   
4  POLYGON ((-106.19771 49.00174, -106.19771 48.9...           3.0   

   weighted_ALAND20  weighted_AWATER20  weighted_STATEFP10  weighted_GEOID10  \
0      1.209867e+09          1112119.0                30.0         3059250.0   
1      1.265276e+07          1100110.0                53.0         5398281.0   
2      4.662729e+08         10084945.0                30.0         3059484.0   
3      1.026204e+09           164299.0                30.0         3059260.0   
4      2.025355e+08           523544.0                30.0         3059256.0   

   weighted_MULTI_STATE  weighted_POP  weighted_PR_HRI  weighted_F_HRI  ...  \
0                   0.0         198.0             20.0             0.5  ...   
1                   0.0        1314.0             91.0             1.0  ...   
2                   0.0         111.0             20.0             0.5  ...   
3                   0.0         110.0             20.0             0.5  ...   
4                   0.0          59.0           -999.0          -999.0  ...   

   weighted_OVERALL_RANK  mode_ZCTA5CE20  mode_AFFGEOID20  mode_GEOID20  \
0                 0.5660           59250   860Z200US59250         59250   
1                 0.6419           98281   860Z200US98281         98281   
2                 0.6977           59484   860Z200US59484         59484   
3                 0.3299           59260   860Z200US59260         59260   
4                 0.5314           59256   860Z200US59256         59256   

   mode_NAME20  mode_LSAD20  mode_STATE  mode_STATE_ABV  mode_ZCTA  highlight  
0        59250           Z5          MT         Montana      59250      False  
1        98281           Z5          WA      Washington      98281      False  
2        59484           Z5          MT         Montana      59484      False  
3        59260           Z5          MT         Montana      59260      False  
4        59256           Z5          MT         Montana      59256      False  

[5 rows x 85 columns]

In [6]:
gdf.columns

Index(['geometry', 'raster_value', 'weighted_ALAND20', 'weighted_AWATER20',
       'weighted_STATEFP10', 'weighted_GEOID10', 'weighted_MULTI_STATE',
       'weighted_POP', 'weighted_PR_HRI', 'weighted_F_HRI', 'weighted_LOW_EMS',
       'weighted_P_NEHD', 'weighted_PR_NEHD', 'weighted_HHB_SCORE',
       'weighted_HHB_RANK', 'weighted_P_CHD', 'weighted_PR_CHD',
       'weighted_F_CHD', 'weighted_P_OBS', 'weighted_PR_OBS', 'weighted_F_OBS',
       'weighted_P_DIABETES', 'weighted_PR_DIABETES', 'weighted_F_DIABETES',
       'weighted_P_COPD', 'weighted_PR_COPD', 'weighted_F_COPD',
       'weighted_P_ASTHMA', 'weighted_PR_ASTHMA', 'weighted_F_ASTHMA',
       'weighted_P_MNTHL', 'weighted_PR_MNTHL', 'weighted_F_MNTHL',
       'weighted_F_SEN_COUNT', 'weighted_SEN_SCORE', 'weighted_SEN_RANK',
       'weighted_P_UNINSUR', 'weighted_PR_UNINSUR', 'weighted_P_POV',
       'weighted_PR_POV', 'weighted_P_UNEMP', 'weighted_PR_UNEMP',
       'weighted_P_NOHSDP', 'weighted_PR_NOHSDP', 'weighted_P_ISO'

In [7]:
hhi_data = pd.read_excel('./HHI_Data/HHI Data 2024 United States.xlsx')

In [8]:
hhi_data.head()

STATEFP10 STATE      STATE_ABV  ZCTA  GEOID10  MULTI_STATE    POP  PR_HRI  \
0         25    MA  Massachusetts  1001  2501001            0  16769      64   
1         25    MA  Massachusetts  1002  2501002            0  29049      67   
2         25    MA  Massachusetts  1003  2501003            0  10372      20   
3         25    MA  Massachusetts  1005  2501005            0   5079      20   
4         25    MA  Massachusetts  1007  2501007            0  14649      58   

   F_HRI  LOW_EMS  ...     P_RENT  PR_RENT   P_OZONE  PR_OZONE  P_PM25  \
0   0.75        0  ...  27.047079   0.6068  0.333333    0.4580     0.0   
1   0.75        0  ...  50.989998   0.9119  0.000000    0.0000     0.0   
2   0.25        1  ...  90.476190   0.9898  0.000000    0.0000     0.0   
3   0.25        0  ...  13.888889   0.2152  0.000000    0.0000     0.0   
4   0.75        0  ...  18.551141   0.3538  0.500000    0.6086     0.0   

   PR_PM25  NBE_SCORE  NBE_RANK  OVERALL_SCORE  OVERALL_RANK  
0      0.0     0.4645    0.5632         0.5703        0.6978  
1      0.0     0.3750    0.3196         0.4838        0.5314  
2      0.0     0.3494    0.2550         0.4246        0.4113  
3      0.0     0.2338    0.0655         0.2391        0.0968  
4      0.0     0.3866    0.3498         0.3194        0.2106  

[5 rows x 75 columns]

In [9]:
hhi_data.columns

Index(['STATEFP10', 'STATE', 'STATE_ABV', 'ZCTA', 'GEOID10', 'MULTI_STATE',
       'POP', 'PR_HRI', 'F_HRI', 'LOW_EMS', 'P_NEHD', 'PR_NEHD', 'HHB_SCORE',
       'HHB_RANK', 'P_CHD', 'PR_CHD', 'F_CHD', 'P_OBS', 'PR_OBS', 'F_OBS',
       'P_DIABETES', 'PR_DIABETES', 'F_DIABETES', 'P_COPD', 'PR_COPD',
       'F_COPD', 'P_ASTHMA', 'PR_ASTHMA', 'F_ASTHMA', 'P_MNTHL', 'PR_MNTHL',
       'F_MNTHL', 'F_SEN_COUNT', 'SEN_SCORE', 'SEN_RANK', 'P_UNINSUR',
       'PR_UNINSUR', 'P_POV', 'PR_POV', 'P_UNEMP', 'PR_UNEMP', 'P_NOHSDP',
       'PR_NOHSDP', 'P_ISO', 'PR_ISO', 'P_ELP', 'PR_ELP', 'P_DISABL',
       'PR_DISABL', 'P_ODW', 'PR_ODW', 'P_AGE65', 'PR_AGE65', 'P_AGE5',
       'PR_AGE5', 'SOCIODEM_SCORE', 'SOCIODEM_RANK', 'P_IMPERV', 'PR_IMPERV',
       'P_TREEC', 'PR_TREEC', 'P_NOVEH', 'PR_NOVEH', 'P_MOBILE', 'PR_MOBILE',
       'P_RENT', 'PR_RENT', 'P_OZONE', 'PR_OZONE', 'P_PM25', 'PR_PM25',
       'NBE_SCORE', 'NBE_RANK', 'OVERALL_SCORE', 'OVERALL_RANK'],
      dtype='object')

In [10]:
def geotiff_to_geodataframe(file_path):
    with rasterio.open(file_path) as src:
        image = src.read(1)
        mask = image != src.nodata
        results = (
            {'properties': {'raster_value': v}, 'geometry': s}
            for i, (s, v) 
            in enumerate(shapes(image, mask=mask, transform=src.transform))
        )
        geoms = list(results)
    
    gdf = gpd.GeoDataFrame.from_features(geoms, crs=src.crs)
    return gdf.to_crs(epsg=4326)

gdf_hr = geotiff_to_geodataframe('./HeatRisk_1_Mercator.tif')

In [11]:
gdf_hr

geometry  raster_value
0      POLYGON ((-94.8817 49.32993, -94.8817 49.30018...           1.0
1      POLYGON ((-95.13265 49.35965, -95.13265 49.225...           3.0
2      POLYGON ((-123.08047 49.00174, -123.08047 48.9...           1.0
3      POLYGON ((-123.05765 49.00174, -123.05765 48.9...           0.0
4      POLYGON ((-118.40349 49.00174, -118.40349 48.9...           1.0
...                                                  ...           ...
15525  POLYGON ((-81.67208 24.69997, -81.67208 24.679...           3.0
15526  POLYGON ((-81.19297 24.7207, -81.19297 24.6999...           2.0
15527  POLYGON ((-81.37549 24.78285, -81.37549 24.762...           3.0
15528  POLYGON ((-81.80896 24.61704, -81.80896 24.575...           2.0
15529  POLYGON ((-81.78615 24.57555, -81.78615 24.534...           3.0

[15530 rows x 2 columns]

# Test adding county and state shapefiles